<a href="https://colab.research.google.com/github/fabioaugusto2102/Business-Technology/blob/main/Trabalho_01_Business_Technology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Previsão de Vendas para a Barba de Elite
Este notebook analisa os dados de estoque e vendas da Barba de Elite, gera previsões utilizando Machine Learning e calcula o Days On Hand (DOH) para um planejamento mais eficiente.


## Importação das bibliotecas

---



Neste bloco realiza-se a importação das bibliotecas que utilizaremos para a implementação do projeto.

---



In [ ]:
# 📌 Importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import plotly.io as pio
import plotly.express as px
from prophet import Prophet



## 📥 Leitura dos arquivos CSV

---


Os arquivos precisam estar pré carregados no sistema do Google Colab. O bloco abaixo realiza a leitura e o carregamento dos dados desses arquivos.

---



In [ ]:
# Carregar os dados
file_estoque = 'estoque_barba_de_elite.csv'
file_vendas = 'compras_barba_de_elite.csv'

df_estoque = pd.read_csv(file_estoque)
df_vendas = pd.read_csv(file_vendas)

# Converter datas
df_vendas['Data e Hora'] = pd.to_datetime(df_vendas['Data e Hora'])
df_vendas['Data'] = df_vendas['Data e Hora'].dt.date

# Visualizar os primeiros dados
df_estoque.head(), df_vendas.head()

(   ID Produto                Nome do Produto  Preço  Quantidade em Estoque
 0           1             Shampoo para barba   39.9                    118
 1           2  Pomada modeladora para cabelo   49.9                    199
 2           3                Óleo para barba   59.9                     76
 3           4                   Cera capilar   44.9                    108
 4           5                  Gel pós-barba   34.9                     14,
    ID Usuário  ID Compra         Data e Hora                        Produto  \
 0        7379     567285 2024-04-11 19:49:00  Pomada modeladora para cabelo   
 1        7379     567285 2024-04-11 19:49:00             Shampoo para barba   
 2        6534     439343 2024-09-30 11:00:00                   Cera capilar   
 3        4919     153892 2024-06-22 21:37:00              Shampoo anticaspa   
 4        4919     153892 2024-06-22 21:37:00                Óleo para barba   
 
    Valor        Data  
 0   49.9  2024-04-11  
 1   39.9  20

## 📊 Análise de Vendas

---

Nesse bloco realiza-se a criação de um gráfico que nos informa o número de vendas diárias. Para conseguir ver o número exato, basta passar o mouse em cima no dia desejado.

---



In [ ]:
# Carregar os dados de vendas
df_vendas = pd.read_csv("compras_barba_de_elite.csv")

# Converter a coluna de data para formato datetime
df_vendas["Data e Hora"] = pd.to_datetime(df_vendas["Data e Hora"])
df_vendas["Data"] = df_vendas["Data e Hora"].dt.date

# Criar a série de vendas diárias
vendas_diarias = df_vendas.groupby("Data").size()

# Criar gráfico interativo de tendência de vendas diárias
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=vendas_diarias.index,
    y=vendas_diarias.values,
    mode="lines+markers",
    name="Vendas Diárias",
    line=dict(color="blue")
))

# Ajustar layout do gráfico
fig.update_layout(
    title="Vendas Diárias",
    xaxis_title="Data",
    yaxis_title="Quantidade de Vendas",
    xaxis=dict(tickangle=-45),
    template="plotly_white",
    hovermode="x unified"
)

# Exibir o gráfico interativo
fig.show()


## 📊 Análise de Vendas por Mês

---

Nesse bloco realiza-se a criação de um gráfico que nos informa o número de vendas mensais.

---



In [ ]:
# Criar colunas de mês e nome do mês
df_vendas["Data e Hora"] = pd.to_datetime(df_vendas["Data e Hora"])  # Garantir que a coluna seja datetime
df_vendas["Mês"] = df_vendas["Data e Hora"].dt.month

# Criar um dicionário para traduzir os meses
meses_traducao = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril", 5: "Maio", 6: "Junho",
    7: "Julho", 8: "Agosto", 9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

df_vendas["Mês"] = df_vendas["Mês"].map(meses_traducao)  # Traduzir os meses para português

# Contar vendas por mês
vendas_por_mes = df_vendas["Mês"].value_counts().reindex(meses_traducao.values())

# Criar gráfico interativo
fig = px.bar(
    x=vendas_por_mes.index,
    y=vendas_por_mes.values,
    text=vendas_por_mes.values,
    labels={"x": "Mês", "y": "Quantidade de Vendas"},
    title="Vendas por Mês",
    color=vendas_por_mes.values,
    color_continuous_scale="Blues"
)

# Ajustar layout
fig.update_traces(textposition="auto")
fig.update_layout(
    xaxis_title="Mês",
    yaxis_title="Quantidade de Vendas",
    template="plotly_white"
)

# Exibir gráfico interativo
fig.show()


## 🏷️ Previsão de Vendas com ML

---

Nesse bloco o código roda um modelo de ML. Essa ML faz uma previsibilidade de venda dos próximos 6 meses.

---


In [ ]:
# Garantir que a coluna de data esteja no formato datetime
df_vendas["Data e Hora"] = pd.to_datetime(df_vendas["Data e Hora"])
df_vendas["Data"] = pd.to_datetime(df_vendas["Data e Hora"].dt.date)

# Agrupar as vendas por dia
df_vendas_diarias = df_vendas.groupby("Data").size().reset_index()
df_vendas_diarias.columns = ["ds", "y"]

# Inicializar e ajustar o modelo Prophet
modelo = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True
)
modelo.fit(df_vendas_diarias)

# Criar DataFrame futuro para 6 meses
futuro = modelo.make_future_dataframe(periods=180)
previsao = modelo.predict(futuro)

# Criar gráfico interativo mais limpo
fig = go.Figure()

# Linha de vendas reais
fig.add_trace(go.Scatter(
    x=df_vendas_diarias["ds"],
    y=df_vendas_diarias["y"],
    mode="lines",
    name="Vendas Reais",
    line=dict(color="#1f77b4", width=2)
))

# Linha de previsão
fig.add_trace(go.Scatter(
    x=previsao["ds"],
    y=previsao["yhat"],
    mode="lines",
    name="Previsão",
    line=dict(color="#ff7f0e", width=2, dash="dash")
))

# Layout clean
fig.update_layout(
    title="Previsão de Vendas para os Próximos 6 Meses (Prophet)",
    xaxis_title="Data",
    yaxis_title="Quantidade de Vendas",
    template="plotly_white",
    hovermode="x unified",
    showlegend=True,
    legend=dict(x=0.01, y=0.99, bgcolor="rgba(255,255,255,0)", bordercolor="rgba(0,0,0,0)")
)

fig.show()


DEBUG:cmdstanpy:input tempfile: /tmp/tmp65qu13yj/2gh8jl8t.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp65qu13yj/0f67ettn.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=23842', 'data', 'file=/tmp/tmp65qu13yj/2gh8jl8t.json', 'init=/tmp/tmp65qu13yj/0f67ettn.json', 'output', 'file=/tmp/tmp65qu13yj/prophet_model9dmoyefv/prophet_model-20250402215922.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:59:22 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:59:22 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


## 📦 Vendas de Produto

---

Nesse bloco realiza-se a leitura de quantidades vendidas e logo em seguida entrega um gráfico com esses dados para uma melhor visualização.

---



In [ ]:
# Contar a quantidade total vendida por item
vendas_por_produto = df_vendas["Produto"].value_counts().reset_index()
vendas_por_produto.columns = ["Produto", "Quantidade Vendida"]

# Exibir os dados
# Assuming display_dataframe_to_user is a function you want to apply:
# If it's in a specific module, import it first (e.g., from my_module import display_dataframe_to_user)
# If it's a standalone function, define it before this cell
# If it's meant to be display, simply use:
display(vendas_por_produto)

# Carregar os dados de vendas
df_vendas = pd.read_csv("compras_barba_de_elite.csv")

# Contar a quantidade total vendida por item
vendas_por_produto = df_vendas["Produto"].value_counts().reset_index()
vendas_por_produto.columns = ["Produto", "Quantidade Vendida"]

# Criar gráfico interativo de vendas por produto
fig = go.Figure()

fig.add_trace(go.Bar(
    x=vendas_por_produto["Produto"],
    y=vendas_por_produto["Quantidade Vendida"],
    marker=dict(color="royalblue"),
    text=vendas_por_produto["Quantidade Vendida"],
    textposition="auto"
))

# Ajustar layout do gráfico
fig.update_layout(
    title="Quantidade Vendida por Produto",
    xaxis_title="Produto",
    yaxis_title="Quantidade Vendida",
    xaxis=dict(tickangle=-45),
    template="plotly_white"
)

# Exibir o gráfico interativo
fig.show()

,Produto,Quantidade Vendida
0,Pomada modeladora para cabelo,1069
1,Gel pós-barba,1069
2,Cera capilar,1066
3,Shampoo para barba,1060
4,Balm para barba,1058
5,Óleo para barba,1051
6,Condicionador para barba,1022
7,Shampoo anticaspa,1017
8,Espuma de barbear,1013
9,Kit completo para barba,1009


## 📊 Análise de Vendas por Dia da Semana


---

Nesse bloco vemos a quantidade de vendas por dia de semana, ajudando na visualização da sazonalidade de vendas dentro de uma semana.

---



In [ ]:
# Carregar os dados de vendas
df_vendas = pd.read_csv("compras_barba_de_elite.csv")

# Converter a coluna de data para formato datetime
df_vendas["Data e Hora"] = pd.to_datetime(df_vendas["Data e Hora"])

# Criar uma coluna com o nome do dia da semana em português
dias_semana_traducao = {
    "Monday": "Segunda-feira",
    "Tuesday": "Terça-feira",
    "Wednesday": "Quarta-feira",
    "Thursday": "Quinta-feira",
    "Friday": "Sexta-feira",
    "Saturday": "Sábado",
    "Sunday": "Domingo"
}

df_vendas["Dia da Semana"] = df_vendas["Data e Hora"].dt.day_name().map(dias_semana_traducao)

# Contar vendas por dia da semana
vendas_por_dia_semana = df_vendas.groupby("Dia da Semana").size()

# Reordenar para começar na segunda-feira
ordem_dias = ["Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sábado", "Domingo"]
vendas_por_dia_semana = vendas_por_dia_semana.reindex(ordem_dias)

# Criar gráfico interativo
fig = go.Figure()

fig.add_trace(go.Bar(
    x=vendas_por_dia_semana.index,
    y=vendas_por_dia_semana.values,
    marker=dict(color="royalblue"),
    text=vendas_por_dia_semana.values,
    textposition="auto"
))

# Ajustar layout do gráfico
fig.update_layout(
    title="Vendas por Dia da Semana",
    xaxis_title="Dia da Semana",
    yaxis_title="Número de Vendas",
    template="plotly_white"
)

# Exibir o gráfico interativo
fig.show()


## 📦 Relação entre Estoque e Vendas

---

Nesse bloco temos um headmap que nos mostra uma relação de vendas por estoque.

---



In [ ]:
# Carregar os dados de vendas e estoque
df_vendas = pd.read_csv("compras_barba_de_elite.csv")
df_estoque = pd.read_csv("estoque_barba_de_elite.csv")

# Calcular a média de vendas diárias por produto
total_dias = df_vendas["Data e Hora"].nunique()
media_vendas_produto = df_vendas.groupby("Produto").size().div(total_dias).reset_index(name="Média de Vendas Diárias")

# Mesclar com a base de estoque
estoque_vendas = df_estoque.merge(media_vendas_produto, left_on="Nome do Produto", right_on="Produto", how="left")
estoque_vendas.drop(columns=["Produto"], inplace=True)

# Calcular DOH (Days On Hand) e Estoque Ideal
estoque_vendas["Days On Hand (DOH)"] = estoque_vendas["Quantidade em Estoque"] / estoque_vendas["Média de Vendas Diárias"]
estoque_vendas["Estoque Ideal (30 dias)"] = estoque_vendas["Média de Vendas Diárias"] * 30

# Criar gráfico interativo de dispersão entre estoque e vendas
fig = px.scatter(
    estoque_vendas,
    x="Quantidade em Estoque",
    y="Média de Vendas Diárias",
    text="Nome do Produto",
    title="Relação entre Estoque e Vendas",
    labels={"Quantidade em Estoque": "Quantidade em Estoque", "Média de Vendas Diárias": "Média de Vendas Diárias"},
    color="Days On Hand (DOH)",  # Cor indicando o DOH
    size="Média de Vendas Diárias",
    hover_name="Nome do Produto",
)

# Ajustes no layout
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color="DarkSlateGrey")))
fig.update_layout(template="plotly_white")

# Exibir gráfico interativo
fig.show()


## 📌 Cálculo de Days On Hand (DOH) e Estoque Ideal

---

Aqui vemos uma análise de DOH (Days on hand) que nos mostra quantos dias levará para vendermos todo nosso estoque atual, baseado nas vendas anteriores.

---



In [ ]:
# Carregar os dados de vendas
df_vendas = pd.read_csv("compras_barba_de_elite.csv")

# Converter a coluna de data para formato datetime
df_vendas["Data e Hora"] = pd.to_datetime(df_vendas["Data e Hora"])
# Creating 'Data' column here to ensure it exists
df_vendas["Data"] = df_vendas["Data e Hora"].dt.date

# Contar o número de dias distintos na base de vendas
total_dias = df_vendas["Data"].nunique()

# Calcular a média de vendas diárias corretamente
media_vendas_produto = df_vendas.groupby('Produto').size().div(total_dias).reset_index(name='Média de Vendas Diárias')

# Mesclar com a base de estoque
estoque_vendas = df_estoque.merge(media_vendas_produto, left_on='Nome do Produto', right_on='Produto', how='left')
estoque_vendas.drop(columns=['Produto'], inplace=True)

# Calcular DOH corretamente
estoque_vendas['Days On Hand (DOH)'] = estoque_vendas['Quantidade em Estoque'] / estoque_vendas['Média de Vendas Diárias']

# Calcular estoque ideal para 30 dias de cobertura
estoque_vendas['Estoque Ideal (30 dias)'] = estoque_vendas['Média de Vendas Diárias'] * 30

# Exibir tabela corrigida
estoque_vendas.head()

,ID Produto,Nome do Produto,Preço,Quantidade em Estoque,Média de Vendas Diárias,Days On Hand (DOH),Estoque Ideal (30 dias)
0,1,Shampoo para barba,39.9,118,2.690355,43.860377,80.710660
1,2,Pomada modeladora para cabelo,49.9,199,2.713198,73.345182,81.395939
2,3,Óleo para barba,59.9,76,2.667513,28.490961,80.025381
3,4,Cera capilar,44.9,108,2.705584,39.917448,81.167513
4,5,Gel pós-barba,34.9,14,2.713198,5.159963,81.395939


## 📊 Análise de Ticket Médio Cliente e Compra

---

Esse bloco nos dá a visualização do TM por cliente e por compra.

---



In [ ]:
# Ticket médio por cliente
ticket_medio_cliente = df_vendas.groupby("ID Usuário")["Valor"].sum().mean()

# Ticket médio por compra
ticket_medio_compra = df_vendas.groupby("ID Compra")["Valor"].sum().mean()

print(f"💰 Ticket Médio por Cliente: R$ {ticket_medio_cliente:.2f}")
print(f"🛒 Ticket Médio por Compra: R$ {ticket_medio_compra:.2f}")


💰 Ticket Médio por Cliente: R$ 249.88
🛒 Ticket Médio por Compra: R$ 158.43


## 📊 Itens Mais Comprados Juntos

---

Esse bloco nos mostra ideias de kits que podem ser criados, por esses são os itens mais comprados juntos em uma única compra.

---



In [ ]:
# Import the necessary module
from itertools import combinations
from collections import Counter # Import Counter from collections

# Criar um dicionário para armazenar os produtos comprados em cada ID de compra
compras_dict = df_vendas.groupby("ID Compra")["Produto"].apply(list)

# Gerar todas as combinações possíveis de produtos
todas_combinacoes = []
for produtos in compras_dict:
    # Now 'combinations' is defined and can be used
    todas_combinacoes.extend(combinations(produtos, 2))

# Contar as combinações mais comuns
comb_count = Counter(todas_combinacoes)
mais_comprados_juntos = comb_count.most_common(5)  # Top 5

# Exibir os produtos mais comprados juntos
print("🔗 Produtos mais comprados juntos:")
for (prod1, prod2), qtd in mais_comprados_juntos:
    print(f"{prod1} + {prod2} → {qtd} vezes")

🔗 Produtos mais comprados juntos:
Balm para barba + Shampoo para barba → 159 vezes
Shampoo anticaspa + Shampoo para barba → 159 vezes
Shampoo para barba + Espuma de barbear → 158 vezes
Cera capilar + Pomada modeladora para cabelo → 156 vezes
Pomada modeladora para cabelo + Espuma de barbear → 152 vezes


## 📊 Dia de Maior Venda

---

Esse bloco nos dá a visualização do dia com maior venda, possibilidando a visibilidade de alguma ação que deu certo ou nos levando a pensar no motivo dessa máxima de vendas.

---



In [ ]:
# Garantir que a coluna Data e Hora seja datetime
df_vendas["Data e Hora"] = pd.to_datetime(df_vendas["Data e Hora"])

# Criar uma coluna apenas com a data (sem horário)
df_vendas["Data"] = df_vendas["Data e Hora"].dt.date

# Encontrar o dia com maior número de vendas
pico_vendas = df_vendas["Data"].value_counts().idxmax()

print(f"📅 Dia com maior número de vendas: {pico_vendas}")


📅 Dia com maior número de vendas: 2024-06-08


## 🔄 Taxa de clientes recorrentes

---

Esse bloco nos dá a taxa de recorrência de clientes , mostrando quantos % dos clientes voltam a comprar nossos produtos.

---



In [ ]:
clientes_com_varias_compras = df_vendas["ID Usuário"].value_counts()
taxa_recorrencia = (clientes_com_varias_compras > 1).mean() * 100
print(f"🔄 Taxa de clientes recorrentes: {taxa_recorrencia:.2f}%")


🔄 Taxa de clientes recorrentes: 94.13%


## 📊 Distribuição da Faixa de Preço

---

Nesse bloco vemos um histograma com a distribuição de feixa de preço mais recorrentes entre as compras.

---



In [ ]:
# Garantir que a coluna de data seja datetime
df_vendas["Data e Hora"] = pd.to_datetime(df_vendas["Data e Hora"])

# Agrupar as compras pelo ID da Compra para calcular o valor total gasto por compra
compras_agrupadas = df_vendas.groupby("ID Compra")["Valor"].sum().reset_index()

# Definir faixas de preço para segmentação
bins = [0, 50, 100, 200, 500, 1000]
labels = ["Até R$50", "R$50-100", "R$100-200", "R$200-500", "Acima de R$500"]

# Classificar cada compra em uma faixa de preço
compras_agrupadas["Faixa de Preço"] = pd.cut(compras_agrupadas["Valor"], bins=bins, labels=labels)

# Contar quantidade de compras por faixa de preço (considerando compras únicas)
faixa_preco = compras_agrupadas["Faixa de Preço"].value_counts().sort_index()

# Criar gráfico interativo
fig = px.bar(
    x=faixa_preco.index,
    y=faixa_preco.values,
    text=faixa_preco.values,
    labels={"x": "Faixa de Preço", "y": "Quantidade de Compras"},
    title="Distribuição de Compras por Faixa de Preço",
    color=faixa_preco.values,
    color_continuous_scale="Blues"
)

# Ajustar layout
fig.update_traces(textposition="auto")
fig.update_layout(
    xaxis_title="Faixa de Preço",
    yaxis_title="Quantidade de Compras",
    template="plotly_white"
)

# Exibir gráfico interativo
fig.show()
